# Meeting August 6

## Challenges: How to optimize shared memory 
### Approach 1
128 KB on Volta GV100 
Double-precision floating point: 64 bit = 8 Bytes

128000/8 = 16000

if pencil-width = 4, then length = 4000 <br>
if pencil-width = 16, then length = 1000


![stencil](graphics/stencils.png)

In this method (https://developer.nvidia.com/blog/finite-difference-methods-cuda-c-part-2/), the pencil length is equivalent to the length of the mesh in one dimension. The largest grid that our operators can work on is restricted by this pencil length. Also with other issues of mapping, the method demonstrated in this paper is not designed to work on a large grid. But we want to work on a large problem.

### Approach 2
Using square tiles for shared memory as used in GPU copy kernels, but they are ineffective for finite difference method.


### Approach 3
Using pencil shapes, but allow multiple "pencils" connected end-to-end to cover one dimension. So in this way, we can avoid the restriction in approach 1. But now here's the problem.

We need two tile dimensions for a 2d pencil: width and height. In method one, we only have one-variable for the pencil-shaped tile: the width of the pencil, because the length is determined by the length of grid in one dimension. In method 2, we also only have one-variable for the square size. But in approach 3, we have two variables. This has issues when determining the block dimensions for launching 2D GPU kernels. </br>




```
## copy_tiled_shared GPU kernel

function copy_tiled_shared!(b, a, ::Val{TILE_DIM}, ::Val{BLOCK_ROWS}) where {TILE_DIM, BLOCK_ROWS}
    N = size(a, 1)
    tidx, tidy = threadIdx().x, threadIdx().y
    i = (blockIdx().x-1) * TILE_DIM + tidx
    j = (blockIdx().y-1) * TILE_DIM + tidy

    tile = @cuStaticSharedMem(eltype(a), (TILE_DIM, TILE_DIM))

    @unroll for k = 0:BLOCK_ROWS:TILE_DIM-1
        if i <= N && (j+k) <= N
            @inbounds tile[tidx, tidy+k] = a[i, j+k]
            
        end
    end


    sync_threads()

    @unroll for k = 0:BLOCK_ROWS:TILE_DIM-1
        if i <= N && (j+k) <= N
            @inbounds b[i, j+k] = tile[tidx, tidy+k]
        end
    end

    nothing
end



## tester function

function tester(fun!, b_ref, d_b, d_a, blockdim, griddim, num_reps, args...)
    @printf("%40s", fun!)
    @cuda threads=blockdim blocks=griddim fun!(d_b, d_a, args...)
    fill!(d_b, 0)

    synchronize()
    t1 = time_ns()
    for i = 1:num_reps
        @cuda threads=blockdim blocks=griddim fun!(d_b, d_a, args...)
    end
    synchronize()
    t2 = time_ns()

    @assert b_ref == Array(d_b)
    nanoseconds = (t2-t1)

    memsize = length(b_ref) * sizeof(eltype(b_ref))
    @printf("%20.2f\n", 2 * memsize * num_reps / nanoseconds)
end



tester(copy_tiled_shared!, a, d_b, d_a, (TILE_DIM, BLOCK_ROWS), griddim, num_reps,Val(TILE_DIM), Val(BLOCK_ROWS))

```


When testing copy_tiled_shared!, we launch 2d GPU kernels, but 2d block dimensions is determined by (TILE_DIM, BLOCK_ROWS), the first variable TILE_DIM is the size of the square tile of shared memory, the other variable BLOCK_ROWS is coupled with this one by this line:

`@unroll for k = 0:BLOCK_ROWS:TILE_DIM-1`

So if we have two different TILE_DIMS because we now have a pencil-shaped memory, we then need to have <br>
TILE_DIM_1, BLOCK_ROWS_1, <br>
TILE_DIM_2, BLOCK_ROWS_2. 

I feel like I can not do this with 2D GPU kernels ... Do I need a 4D GPU kernel? I thought this was trivial just by merging two different versions that I could easily finished it earlier this week, but I havn't found a nice way yet. Which is before even when I start to consider avoiding bank conflicts by doing somethis like this
`tile = @cuStaticSharedMem(eltype(a), (TILE_DIM+1, TILE_DIM))` 



### Approach 4
In approach 3, if we still want a pencil shape but want reduced numbers of variables, we can set TILE_DIM_1 / TILE_DIM_2 = const. By Directly setting this const to be something like 32. We fix the shape of the pencil, leaving either TILE_DIM_1 or TILE_DIM_2 to be an independent variable. This is something similar to having a square tile where you only have one independent variable, but then you devide this square tile into multiple different tiles. I am not sure if this is also compatible with 2D GPU block_dims with minor modifications to (TILE_DIM, BLOCK_ROWS).


## What I could do now?

1. Continue designing shared memory mapping scheme:
But I have to design a GPU kernel to deal with lots of tricky variables and how they are associated

The shapes of the mesh    (Nx, Ny) <br>
The shapes of GPU blocks   (BLOCK_DIMs)
The shapes of pencils for shared memory (TILE_DIMs)

in a 2D GPU kernel


2. Forget about using shared memory and move on implementing the rest operators, since our method is already fast enough (Memory Through-put is close to 50% of probably optimized ones )


3. Use either approach 1 or approach 2 with limitations. 

limitations for approach 1: ideal for finite difference, but has the limitation for the maximum size of mesh we can work with (probably only in non-hybridized method)

limitations for approach 2: it can work with any size, it should be better than not using shared memory at all, but it's definitely not ideal for finite difference

## What else I did this week
Mostly I was reading blogs by Mark Harris and understanding memory mechanisms on GPUs.

https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/ <br>
https://developer.nvidia.com/blog/efficient-matrix-transpose-cuda-cc/ (actually GPU code in Julia written by Jeremy and Lucas are modified from this blog as mentioned at the beginning https://gitlab.nps.edu/ma4261_2002/website/-/blob/master/topic02/transpose.jl)  <br>    
https://developer.nvidia.com/blog/cuda-pro-tip-increase-performance-with-vectorized-memory-access/

Some other blogs explaning GPU concepts associated to Mark's posts
Bank conflicts: http://cuda-programming.blogspot.com/2013/02/bank-conflicts-in-shared-memory-in-cuda.html

## Prototype of Proposed GPU tiling scheme

```
function D2x_GPU_v4(d_u, d_y, Nx, Ny, h, ::Val{TILE_DIM1}, ::Val{TILE_DIM2}) where {TILE_DIM1, TILE_DIM2}
	tidx = threadIdx().x
	tidy = threadIdx().y

	i = (blockIdx().x - 1) * TILE_DIM1 + tidx
	j = (blockIdx().y - 1) * TILE_DIM2 + tidy

	global_index = i + (j-1)*Ny

	# i = (blockIdx().x - 1) * TILE_DIM + threadIdx().x
	tile = @cuStaticSharedMem(eltype(d_u),(TILE_DIM1,TILE_DIM2))

	k = tidx
	l = tidy

	if k <= TILE_DIM1 && l <= TILE_DIM2 && global_index <= Nx*Ny
		tile[k,l] = d_u[global_index]
	end

	sync_threads()


	if k <= TILE_DIM1 && l <= TILE_DIM2 && global_index <= Nx*Ny
		d_y[global_index] = tile[k,l]
	end

	sync_threads()


	# N = Nx*Ny
	# # d_y = zeros(N)
	# if tidx <= Ny
	# 	d_y[tidx] = (d_u[tidx] - 2 * d_u[Ny + tidx] + d_u[2*Ny + tidx]) / h^2
	# end
	#
	# if Ny+1 <= tidx <= N-Ny
	# 	d_y[tidx] = (d_u[tidx - Ny] - 2 * d_u[tidx] + d_u[tidx + Ny]) / h^2
	# end
	#
	#
	# if N-Ny+1 <= tidx <= N
	# 	d_y[tidx] = (d_u[tidx - 2*Ny] -2 * d_u[tidx - Ny] + d_u[tidx]) / h^2
	# end
	#
	# sync_threads()

	nothing
end



function tester_D2x_v4(Nx)
	Ny = Nx
	h = 1/Nx
	TILE_DIM_1 = 2
	TILE_DIM_2 = 16

	d_u = CuArray(randn(Nx*Ny))
	d_y = similar(d_u)

	griddim = (div(Nx,TILE_DIM_1) + 1, div(Ny,TILE_DIM_2) + 1)
	blockdim = (TILE_DIM_1,TILE_DIM_2)

	@cuda threads=blockdim blocks=griddim D2x_GPU_v4(d_u,d_y,Nx,Ny,h,Val(TILE_DIM_1),Val(TILE_DIM_2))
	@show Array(d_u) ≈ Array(d_y)
	return nothing
end

```

This is the result of merging C++ GPU kernel code with Jeremy's Julia tiled matrix copying code.


1. I used global_index from C++ code to access data from d_u which is a vector rather than a matrix in Jeremy's code
2. For mapping between GPU blocks and threads to elements in d_u, I used something similar in Jeremy's code, which is more intuitive and can work on larger problem.
3. For tiling, I used pencil tiling, but it has two variables instead of just one in C++ code. I tested different pencil shapes and they all worked
4. Currently I only have vector copying code with 2D GPU kernel to test the tiling mechanism. I need to incorporate the snippet that is commented out in D2x_GPU_v4() to make it a real differential operator. Or I can use something similar to C++ code, which is easier for determining three cases I guess. (Or maybe not because our SBP operators are different)

```
  if (j < 4) {
    s_f[sj-4][si]  = s_f[sj+my-5][si];
    s_f[sj+my][si] = s_f[sj+1][si];
  }

  __syncthreads();

  df[globalIdx] = 
    ( c_ay * ( s_f[sj+1][si] - s_f[sj-1][si] )
    + c_by * ( s_f[sj+2][si] - s_f[sj-2][si] )
    + c_cy * ( s_f[sj+3][si] - s_f[sj-3][si] )
    + c_dy * ( s_f[sj+4][si] - s_f[sj-4][si] ) );
```

5. I haven't tested the memory through-put

